# FMRI DATA ANALYSIS - NARPS DATASET BY TEAM 98BT

This notebook reproduce the analysis made by team **98BT** for the NARPS experiment (Botvinik-Nezer & al, 2020). 

**To use this notebook :** 
- Download fMRIprep data available [**here**](https://openneuro.org/datasets/ds001734/versions/1.0.4) or in [**datalad**](http://datasets.datalad.org/?dir=/workshops/nih-2017/ds001734).
- Modify the paths:
    - **exp_dir**: directory where the ds001734-download repository is stored
    - **result_dir**: directory where the intermediate and final repositories will be stored
    - **working_dir**: name of the directory where intermediate results will be stored
    - **output_dir**: name of the directory where final results will be stored

Botvinik-Nezer, R., Holzmeister, F., Camerer, C.F., Dreber, A., Huber, J., Johannesson, M., Kirchler, M., Iwanir, R., Mumford, J.A., ..., Nichols, T.E., Poldrack, R.A., Schonberg, T. (2020). Variability in the analysis of a single neuroimaging dataset by many teams. Nature. https://doi.org/10.1038/s41586-020-2314-9.

In [1]:
# Set the path to matlab 
## NECESSARY TO USE SPM
from nipype.interfaces import spm
matlab_cmd = '/local/egermani/spm12/run_spm12.sh /usr/local/MATLAB/R2018a/bin/matlab script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd)
from narps_open.pipelines import team_98BT

In [5]:
team_ID = "98BT"
# Important directories
## exp_dir : where the data are stored (where the ds001734 directory is stored)
exp_dir = '../data/original'

## result_dir : where the intermediate and final results will be store
result_dir = "../data/derived/reproduced"

## working_dir : where the intermediate outputs will be store
working_dir = f"NARPS-{team_ID}-reproduced/intermediate_results"

## output_dir : where the final results will be store
output_dir = f"NARPS-{team_ID}-reproduced"

In [6]:
dir_list = os.listdir(exp_dir)

# Subject list (to which we will do the analysis)
subject_list = []
for dirs in dir_list:
    if dirs[0:3] == 'sub':
        subject_list.append(dirs[-3:])

In [7]:
# Run to use for this analysis
run_list = ['01', '02', '03', '04']

#TR
with open(opj(exp_dir, 'task-MGT_bold.json'), 'rt') as fp:
    task_info = json.load(fp)
TR = task_info['RepetitionTime']
ST = task_info['SliceTiming']
N = len(ST)
TA = TR/N
ET = task_info["EffectiveEchoSpacing"]
total_readout_time = N * ET

#FWHM to smooth (team chose a kernel of 8mm for smoothing)
fwhm = 8

## Dartel workflow

In [ ]:
dartel_wf = team_98BT.get_dartel_template_wf(exp_dir, result_dir, working_dir, output_dir, subject_list)

dartel_wf.run()

## Preprocessing 

In [ ]:
preprocessing = team_98BT.get_preprocessing(exp_dir, result_dir, working_dir, output_dir,
                                                 subject_list, run_list, fwhm, N, ST, TA, TR, total_readout_time)

preprocessing.run()

## Subject level analysis

In [ ]:
l1_analysis = team_98BT.get_l1_analysis(subject_list, TR, run_list, exp_dir, result_dir, 
                                             working_dir, output_dir)

l1_analysis.run()

## Group level analysis

In [ ]:
l2_analysis_equalRange = team_98BT.get_l2_analysis(subject_list, n_sub, contrast_list,
                                                        'equalRange', exp_dir, result_dir, working_dir, output_dir)

l2_analysis_equalIndiff = team_98BT.get_l2_analysis(subject_list, n_sub, contrast_list,
                                                        'equalIndifference', exp_dir, result_dir, 
                                                         working_dir, output_dir)

l2_analysis_groupComp = team_98BT.get_l2_analysis(subject_list, n_sub, contrast_list,
                                                        'groupComp', exp_dir, result_dir, working_dir, output_dir)

In [ ]:
l2_analysis_equalRange.run()

In [ ]:
l2_analysis_equalIndiff.run()

In [ ]:
l2_analysis_groupComp.run()

In [ ]:
team_98BT.reorganize_results(result_dir, output_dir, n_sub, team_ID)